In [1]:
import pymssql
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import datetime

conn= pymssql.connect(server="103.22.220.149", user="sa", password="yonsei2018!", port=1435)

stmt = "select distinct * from [REDCDM].[dbo].[condition_occurrence_100] order by person_id, condition_concept_id, condition_start_date"

c = conn.cursor()
c.execute(stmt)
data= c.fetchall()
#len(data)


########################################################################################    

## group by 적용 값 - 주작업 
res=[]
res2=[]
for idx, i in enumerate( range(1,len(data)) ):
    
    if data[i-1][1]==data[i][1] and data[i-1][2]==data[i][2]:  # person_id, concept_id가 묶인다면,  3=start, 4=end
        a=data[i-1][3]
        b=data[i-1][4]
        a=datetime.datetime.strptime(a, "%Y-%m-%d").date()
        b=datetime.datetime.strptime(b, "%Y-%m-%d").date()
        res.append([data[i-1][1],data[i-1][2],a,b])
    
    else:
        a=data[i-1][3]
        b=data[i-1][4]
        a=datetime.datetime.strptime(a, "%Y-%m-%d").date()
        b=datetime.datetime.strptime(b, "%Y-%m-%d").date()
        res.append([data[i-1][1],data[i-1][2],a,b])
        
        res2.append(res)
        res=[]
#print(res2)



####################################################################################################33
## TEST ##

for idx,i in enumerate(res2): # 0=person_id, 2=concept_id, 3=start, 4=end
    print("TEST:",i[0][2]-i[1][3])
    break
            


TEST: -25 days, 0:00:00


In [4]:
print(res2)

[[['1015944', '134304', datetime.date(2018, 8, 23), datetime.date(2018, 8, 23)], ['1015944', '134304', datetime.date(2018, 9, 17), datetime.date(2018, 9, 17)]], [['1015944', '136934', datetime.date(2018, 10, 12), datetime.date(2018, 10, 12)]], [['1015944', '138388', datetime.date(2018, 8, 29), datetime.date(2018, 8, 29)], ['1015944', '138388', datetime.date(2018, 9, 27), datetime.date(2018, 9, 27)], ['1015944', '138388', datetime.date(2018, 10, 10), datetime.date(2018, 10, 10)], ['1015944', '138388', datetime.date(2018, 10, 11), datetime.date(2018, 10, 11)], ['1015944', '138388', datetime.date(2018, 10, 15), datetime.date(2018, 10, 15)], ['1015944', '138388', datetime.date(2018, 11, 7), datetime.date(2018, 11, 7)]], [['1015944', '99999', datetime.date(2018, 9, 17), datetime.date(2018, 9, 17)], ['1015944', '99999', datetime.date(2018, 10, 10), datetime.date(2018, 10, 10)], ['1015944', '99999', datetime.date(2018, 10, 11), datetime.date(2018, 10, 11)], ['1015944', '99999', datetime.date(

In [3]:

############### 최 종 완 성 :  수정 derived   2019 10 16 ######################################################

param1=[]
param2=[]
param3=[]
param4=[]
param6=[]
param26=[]



for idx, lis in enumerate(res2):    
    if len(lis)==1:  # 1개 행만 갖는 경우   ########### OK   
        person_id1=lis[0][0] 
        condition_concept_id1 = lis[0][1]
        condition_start_date1 = lis[0][2]
        condition_end_date1 = lis[0][3]
        dateCOUNT = 1
        param6.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])
        
        
        
        
        
    else:    # 여러개 행일 때 (이게 대부분)
        cnt=1
        start=lis[0]
       
        
        for idx2 in range(1,len(lis)):
            
            if (lis[idx2-1][3]-lis[idx2][2]).days >= -30 : # 첫행과 30일 차이 안나면                
                cnt+=1
               
                person_id1=start[0]         
                condition_concept_id1 = start[1]
                condition_start_date1 = start[2]  ######################## 30일 차이 안나니까 처음 start를 계속 사용할 것 
                condition_end_date1 = lis[idx2][3]
                dateCOUNT = cnt
                param1.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])

                if idx2==len(lis)-1:
                    #param6.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])  ########## OK
                    param6.append(param1[-1])
                    
                    
                


            else:   # 첫행과 30일 차이 나면!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                
                start=lis[idx2]               
                
                if idx2==1:  # lis의 첫 행 부터 30일 차이 나는 경우 
                    
                    if len(lis)==2: # 행이 2개만 있을 때                                  
                        param6.append([lis[0][0],lis[0][1],lis[0][2],lis[0][3],1])
                        param6.append([lis[1][0],lis[1][1],lis[1][2],lis[1][3],1])  ############### ok                       
  


                else: # 30일 차이 안나는걸로 돌다가 이리로 걸려들어온 경우   
        
                    if param1==[]:  # idx-1 : idx : idx+1  이렇게 인접한 날짜 모두 30일 이상 떨어졌을 때, 여기로 오게 됨 
                        if idx2+1!=len(lis):
                            if (lis[idx2][3]-lis[idx2+1][2]).days < -30: # 연속해서 30일 차이 날 경우!!!!!!!!!!!!!!!!!!!
                                param6.append([lis[idx2][0],lis[idx2][1],lis[idx2][2],lis[idx2][3],1])
                        else:
                            pass # 알아서 상위 30일 비교문으로. 이미 start를 선언해놨기 때문에 문제없음 
                    
                    
                    else:  
                        param6.append(param1[-1])  # 앞서 축적한 데이터의 맨 마지막 값만 param6에 집어넣는다.
                        param1=[]  # 축적된 값 다 날리기. 30일 연속이 끊어짐 
                        cnt=1 # cnt 초기화 

                        if idx2==len(lis):  # 30일 차이나는데, 마지막 행이면 그냥 param6에 삽입하고 끝 
                            param6.append([lis[idx2][0],lis[idx2][1],lis[idx2][2],lis[idx2][3],1])     
                            
                        else:  # 앞에 축적된 값 이어서 30일 바로 끊어지는 값을 넣어줘야 함 
                            
                            if idx2+1!=len(lis):
                                if (lis[idx2][3]-lis[idx2+1][2]).days < -30: # 그 다음 이어지는 값하고도 30일 차이가 나면
                                    param6.append([lis[idx2][0],lis[idx2][1],lis[idx2][2],lis[idx2][3],1])

                    
param6
                       


[['1015944',
  '134304',
  datetime.date(2018, 8, 23),
  datetime.date(2018, 9, 17),
  2],
 ['1015944',
  '136934',
  datetime.date(2018, 10, 12),
  datetime.date(2018, 10, 12),
  1],
 ['1015944',
  '138388',
  datetime.date(2018, 8, 29),
  datetime.date(2018, 11, 7),
  6],
 ['1015944',
  '99999',
  datetime.date(2018, 9, 17),
  datetime.date(2018, 11, 7),
  9],
 ['1016235',
  '4253197',
  datetime.date(2018, 8, 17),
  datetime.date(2018, 8, 23),
  2],
 ['1016235',
  '99999',
  datetime.date(2018, 8, 17),
  datetime.date(2018, 8, 23),
  2],
 ['1040164',
  '36713547',
  datetime.date(2018, 12, 26),
  datetime.date(2018, 12, 26),
  1],
 ['1040164',
  '99999',
  datetime.date(2018, 10, 8),
  datetime.date(2018, 10, 23),
  8],
 ['1040164',
  '99999',
  datetime.date(2018, 12, 20),
  datetime.date(2018, 12, 20),
  4],
 ['1109886',
  '192273',
  datetime.date(2018, 7, 4),
  datetime.date(2018, 7, 11),
  3],
 ['1109886',
  '4151134',
  datetime.date(2018, 7, 4),
  datetime.date(2018, 7, 11),


In [ ]:
# 집어넣어야지 DB에 

import pymssql

conn= pymssql.connect(server="103.22.220.149", user="sa", password="yonsei2018!", port=1435)
c = conn.cursor()


# group by 로 작업한거라서 중간중간 2중 list가 끼어있음.
param6_1=[]
param6_2=[]

for i in param6:
    if len(i)==5:                
        param6_1.append(i)
    else:
        print('else:',i)
        break
        
f=param6_1


cnt=0
for i in f:           
    i1=str(i[0])   
    i2=str(i[1])
    i3=str(i[2])
    i4=str(i[3])
    i5=str(i[4])
    
    cnt+=1
    #c.execute("insert into [MEN_1].[dbo].[condition_era](person_id, condition_concept_id, condition_era_start_date, condition_era_end_date, condition_occurrence_count) values(%s, %s, %s, %s, %s)", (i1,i2,i3,i4,i5) )
    c.execute("insert into [REDCDM].[dbo].[drug_era](person_id, drug_concept_id, drug_era_start_date, drug_era_end_date, drug_exposure_count) values(%s, %s, %s, %s, %s)", (i1,i2,i3,i4,i5) )
    
    if cnt%100==0:
        conn.commit()
        print('  ',cnt,"good",end='')
        
conn.commit()
conn.close()



In [ ]:
################################  1번에 해결 코딩 중  #######################################3  옛 날 꺼 !!!!!!!!!!!!!!!
##########!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!##########################################

param1=[]
param6=[]
param26=[]
cnt=1


for idx, lis in enumerate(res2):
    
    if len(lis)==1:  # 1개 행만 갖는 경우   ########### OK 
        person_id1=lis[0][0]        
        condition_concept_id1 = lis[0][1]
        condition_start_date1 = lis[0][2]
        condition_end_date1 = lis[0][3]
        dateCOUNT = 1
        param6.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])
            
    else:    # 여러개 행일 때 (이게 대부분)
        start=lis[0]
        cnt=1

        for idx2 in range(1,len(lis)):

            if (lis[idx2-1][3]-lis[idx2][2]).days >= -30 : # 30일 차이 안나면
                cnt+=1
                person_id1=lis[idx2-1][0]                
                condition_concept_id1 = lis[idx2-1][1]
                condition_start_date1 = lis[idx2-1][2]
                condition_end_date1 = lis[idx2][3]
                dateCOUNT = cnt
                param1.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])

                if idx2==len(lis)-1:
                    param6.append([person_id1, condition_concept_id1, condition_start_date1, condition_end_date1, dateCOUNT])



            else: 
                
                if idx2==1:  # 첫 행 부터 30일 차이 나는 경우 
                    
                    if len(lis)==2: # 행이 2개만 있을 때                                  
                        param6.append([lis[0][0],lis[0][1],lis[0][2],lis[0][3],1])
                        param6.append([lis[1][0],lis[1][1],lis[1][2],lis[1][3],1])
                    
                else:  
                    if param1==[]:
                        param6.append([start[0], start[1],start[2],start[3],1])
                        start=lis[idx2]

                        if (idx) == len(lis)-1:
                            param6.append(lis[idx2])
                            param6[-1].extend(str(1))

                    else:
                        #print(param1[-1])
                        param6.append(param1[-1])
                        start=lis[idx2]
                        cnt=1
                        param1=[]
                        start=lis[idx2]

                        if idx2 == len(lis)-1:
                            param6.append(lis[idx2])
                            param6[-1].extend(str(1))


param6
